In [195]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import TimeSeriesSplit
from scipy import stats
from collections import Counter
from sklearn.metrics import accuracy_score

In [196]:
weather = pd.read_csv("../Tan - Random Forest/hanoiweather.csv")

In [197]:
from datetime import datetime

def time_to_minutes(time_str):
    # Định nghĩa các định dạng thời gian khác nhau
    formats = ["%Y-%m-%dT%H:%M:%S", "%Y-%m-%d %H:%M:%S"]
    
    for fmt in formats:
        try:
            # Cố gắng phân tích chuỗi thời gian với từng định dạng
            time_obj = datetime.strptime(time_str, fmt)
            return time_obj.hour * 60 + time_obj.minute
        except ValueError:
            # Nếu định dạng không khớp, thử định dạng tiếp theo
            continue
    
    # Nếu không có định dạng nào khớp, in lỗi
    raise ValueError(f"Thời gian '{time_str}' không khớp với bất kỳ định dạng nào đã biết")

# Biến đổi thời gian của sunrise và sunset
weather['sunrise_minutes'] = weather['sunrise'].apply(time_to_minutes)
weather['sunset_minutes'] = weather['sunset'].apply(time_to_minutes)

In [198]:
weather

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations,sunrise_minutes,sunset_minutes
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,NaN,1990-01-01T06:33:41,1990-01-01T17:26:18,0.14,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,393,1046
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,NaN,1990-01-02T06:34:00,1990-01-02T17:26:56,0.18,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1046
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,NaN,1990-01-03T06:34:18,1990-01-03T17:27:34,0.22,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1047
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,NaN,1990-01-04T06:34:35,1990-01-04T17:28:12,0.25,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1048
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,NaN,1990-01-05T06:34:51,1990-01-05T17:28:51,0.29,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12534,Hà Nội,2024-04-26,36.0,27.0,30.9,47.9,31.0,39.6,26.9,80.8,...,60.0,2024-04-26T05:29:23,2024-04-26T18:19:42,0.58,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,VVNB,329,1099
12535,Hà Nội,2024-04-27,40.0,27.0,32.2,44.7,31.2,38.6,24.7,69.9,...,30.0,2024-04-27T05:28:42,2024-04-27T18:20:04,0.61,Clear,Clear conditions throughout the day.,clear-day,"48820099999,48823099999,48825099999,4883109999...",328,1100
12536,Hà Nội,2024-04-28,35.0,27.0,30.4,50.8,30.5,39.4,27.0,82.7,...,60.0,2024-04-28T05:28:02,2024-04-28T18:20:27,0.65,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"48820099999,48823099999,48825099999,4883109999...",328,1100
12537,Hà Nội,2024-04-29,34.0,28.0,30.2,49.3,34.4,39.4,27.4,85.3,...,75.0,2024-04-29T05:27:22,2024-04-29T18:20:49,0.68,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,VVNB,327,1100


In [199]:
# Tạo cột mới có giá trị là nhiệt độ cao nhất của ngày hôm sau 
weather['tempmax_next'] = weather['tempmax'].shift(-1)

In [200]:
weather.head(5)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunset,moonphase,conditions,description,icon,stations,sunrise_minutes,sunset_minutes,tempmax_next
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,1990-01-01T06:33:41,1990-01-01T17:26:18,0.14,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,393,1046,17.4
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,1990-01-02T06:34:00,1990-01-02T17:26:56,0.18,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1046,18.2
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,1990-01-03T06:34:18,1990-01-03T17:27:34,0.22,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1047,19.6
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,1990-01-04T06:34:35,1990-01-04T17:28:12,0.25,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1048,18.1
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,1990-01-05T06:34:51,1990-01-05T17:28:51,0.29,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1048,17.7


In [201]:
# Xử lý dữ liệu khuyết do hàng cuối sẽ là NaN => bỏ dòng cuối
weather = weather[:-1]

In [202]:
weather

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunset,moonphase,conditions,description,icon,stations,sunrise_minutes,sunset_minutes,tempmax_next
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,1990-01-01T06:33:41,1990-01-01T17:26:18,0.14,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,393,1046,17.4
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,1990-01-02T06:34:00,1990-01-02T17:26:56,0.18,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1046,18.2
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,1990-01-03T06:34:18,1990-01-03T17:27:34,0.22,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1047,19.6
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,1990-01-04T06:34:35,1990-01-04T17:28:12,0.25,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1048,18.1
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,1990-01-05T06:34:51,1990-01-05T17:28:51,0.29,Overcast,Cloudy skies throughout the day.,cloudy,48820099999,394,1048,17.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12533,Hà Nội,2024-04-25,34.0,25.0,28.9,45.6,25.0,34.5,25.9,84.7,...,2024-04-25T05:30:05,2024-04-25T18:19:20,0.54,"Rain, Partially cloudy",Partly cloudy throughout the day with early mo...,rain,"48820099999,48823099999,48825099999,4883109999...",330,1099,36.0
12534,Hà Nội,2024-04-26,36.0,27.0,30.9,47.9,31.0,39.6,26.9,80.8,...,2024-04-26T05:29:23,2024-04-26T18:19:42,0.58,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,VVNB,329,1099,40.0
12535,Hà Nội,2024-04-27,40.0,27.0,32.2,44.7,31.2,38.6,24.7,69.9,...,2024-04-27T05:28:42,2024-04-27T18:20:04,0.61,Clear,Clear conditions throughout the day.,clear-day,"48820099999,48823099999,48825099999,4883109999...",328,1100,35.0
12536,Hà Nội,2024-04-28,35.0,27.0,30.4,50.8,30.5,39.4,27.0,82.7,...,2024-04-28T05:28:02,2024-04-28T18:20:27,0.65,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"48820099999,48823099999,48825099999,4883109999...",328,1100,34.0


In [203]:
# Mã hoá Conditions vì là dữ liệu phân loại 
conditions_encoded = pd.get_dummies(weather['conditions'], prefix='cond')
weather= pd.concat([weather, conditions_encoded], axis=1)

In [204]:
weather

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,stations,sunrise_minutes,sunset_minutes,tempmax_next,cond_Clear,cond_Overcast,cond_Partially cloudy,cond_Rain,"cond_Rain, Overcast","cond_Rain, Partially cloudy"
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,48820099999,393,1046,17.4,False,True,False,False,False,False
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,48820099999,394,1046,18.2,False,True,False,False,False,False
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,48820099999,394,1047,19.6,False,True,False,False,False,False
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,48820099999,394,1048,18.1,False,True,False,False,False,False
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,48820099999,394,1048,17.7,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12533,Hà Nội,2024-04-25,34.0,25.0,28.9,45.6,25.0,34.5,25.9,84.7,...,"48820099999,48823099999,48825099999,4883109999...",330,1099,36.0,False,False,False,False,False,True
12534,Hà Nội,2024-04-26,36.0,27.0,30.9,47.9,31.0,39.6,26.9,80.8,...,VVNB,329,1099,40.0,False,False,True,False,False,False
12535,Hà Nội,2024-04-27,40.0,27.0,32.2,44.7,31.2,38.6,24.7,69.9,...,"48820099999,48823099999,48825099999,4883109999...",328,1100,35.0,True,False,False,False,False,False
12536,Hà Nội,2024-04-28,35.0,27.0,30.4,50.8,30.5,39.4,27.0,82.7,...,"48820099999,48823099999,48825099999,4883109999...",328,1100,34.0,False,False,True,False,False,False


In [205]:
# Lựa chọn các loại đặc trưng độ ẩm, lớp phủ mây, tốc độ gió,  áp suất mực nước biển
features = ['tempmax', 'tempmin', 'humidity', 'cloudcover', 'windspeed', 'sealevelpressure', 'moonphase', 'visibility', 'winddir', 'feelslikemax' ,'feelslikemin','feelslike', 'dew', 'temp', 'windgust', 'sunrise_minutes', 'sunset_minutes', ]
X = weather[features]
y = weather['tempmax_next']

In [206]:
#Thêm condition vào features 
condition_features = [col for col in weather.columns if 'cond_' in col]
features.extend(condition_features)

In [207]:
features

['tempmax',
 'tempmin',
 'humidity',
 'cloudcover',
 'windspeed',
 'sealevelpressure',
 'moonphase',
 'visibility',
 'winddir',
 'feelslikemax',
 'feelslikemin',
 'feelslike',
 'dew',
 'temp',
 'windgust',
 'sunrise_minutes',
 'sunset_minutes',
 'cond_Clear',
 'cond_Overcast',
 'cond_Partially cloudy',
 'cond_Rain',
 'cond_Rain, Overcast',
 'cond_Rain, Partially cloudy']

In [208]:
weather.head(5)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,stations,sunrise_minutes,sunset_minutes,tempmax_next,cond_Clear,cond_Overcast,cond_Partially cloudy,cond_Rain,"cond_Rain, Overcast","cond_Rain, Partially cloudy"
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,48820099999,393,1046,17.4,False,True,False,False,False,False
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,48820099999,394,1046,18.2,False,True,False,False,False,False
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,48820099999,394,1047,19.6,False,True,False,False,False,False
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,48820099999,394,1048,18.1,False,True,False,False,False,False
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,48820099999,394,1048,17.7,False,True,False,False,False,False


In [209]:
def entropy(s):
    values, counts = np.unique(s, return_counts=True)
    probabilities = counts / counts.sum()
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

In [210]:
tempmax_entropy = entropy(weather['tempmax'])
print("Entropy of tempmax:", tempmax_entropy)

Entropy of tempmax: 7.015482289848042


In [211]:
X = weather[features]
y = weather['tempmax_next']

In [212]:
print("Số lượng giá trị khuyết trong X:")
print(X.isnull().sum())
print("\nSố lượng giá trị khuyết trong y:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong X:
tempmax                          57
tempmin                          57
humidity                         57
cloudcover                       57
windspeed                         0
sealevelpressure                 99
moonphase                         0
visibility                       57
winddir                          78
feelslikemax                     57
feelslikemin                     57
feelslike                        57
dew                              57
temp                             57
windgust                       9059
sunrise_minutes                   0
sunset_minutes                    0
cond_Clear                        0
cond_Overcast                     0
cond_Partially cloudy             0
cond_Rain                         0
cond_Rain, Overcast               0
cond_Rain, Partially cloudy       0
dtype: int64

Số lượng giá trị khuyết trong y:
57


In [213]:
weather_clean = weather.dropna(subset=features + ['tempmax_next'])
X = weather_clean[features]
y = weather_clean['tempmax_next']

In [214]:
print("Số lượng giá trị khuyết trong X:")
print(X.isnull().sum())
print("\nSố lượng giá trị khuyết trong y:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong X:
tempmax                        0
tempmin                        0
humidity                       0
cloudcover                     0
windspeed                      0
sealevelpressure               0
moonphase                      0
visibility                     0
winddir                        0
feelslikemax                   0
feelslikemin                   0
feelslike                      0
dew                            0
temp                           0
windgust                       0
sunrise_minutes                0
sunset_minutes                 0
cond_Clear                     0
cond_Overcast                  0
cond_Partially cloudy          0
cond_Rain                      0
cond_Rain, Overcast            0
cond_Rain, Partially cloudy    0
dtype: int64

Số lượng giá trị khuyết trong y:
0


In [215]:
# Kiểm tra kiểu dữ liệu của các cột trong `X`
print("\nKiểu dữ liệu của các cột trong X:")
print(X.dtypes)



Kiểu dữ liệu của các cột trong X:
tempmax                        float64
tempmin                        float64
humidity                       float64
cloudcover                     float64
windspeed                      float64
sealevelpressure               float64
moonphase                      float64
visibility                     float64
winddir                        float64
feelslikemax                   float64
feelslikemin                   float64
feelslike                      float64
dew                            float64
temp                           float64
windgust                       float64
sunrise_minutes                  int64
sunset_minutes                   int64
cond_Clear                        bool
cond_Overcast                     bool
cond_Partially cloudy             bool
cond_Rain                         bool
cond_Rain, Overcast               bool
cond_Rain, Partially cloudy       bool
dtype: object


In [216]:
# Thống kê mô tả cho `X`
print("\nThống kê mô tả cho X:")
print(X.describe())

# Thống kê mô tả cho `y`
print("\nThống kê mô tả cho y:")
print(y.describe())


Thống kê mô tả cho X:
           tempmax      tempmin     humidity   cloudcover    windspeed  \
count  3479.000000  3479.000000  3479.000000  3479.000000  3479.000000   
mean     28.240126    21.799310    78.294108    63.975481    17.473124   
std       5.878130     4.883595    10.900963    23.116869     5.230512   
min       8.000000     5.900000    31.100000     0.000000     6.500000   
25%      24.000000    18.000000    72.100000    49.800000    13.900000   
50%      29.000000    23.000000    79.700000    67.200000    16.600000   
75%      33.000000    26.000000    85.800000    82.100000    20.500000   
max      41.700000    31.000000   100.000000   100.000000    50.000000   

       sealevelpressure    moonphase   visibility      winddir  feelslikemax  \
count       3479.000000  3479.000000  3479.000000  3479.000000   3479.000000   
mean        1011.274619     0.483587     8.450474   125.456137     32.084909   
std            7.139166     0.288407     1.797434    97.831172      9.

In [217]:
# Phân chia tập dữ liệu
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Kích thước tập huấn luyện:", X_train.shape, y_train.shape)
print("Kích thước tập kiểm thử:", X_test.shape, y_test.shape)

Kích thước tập huấn luyện: (2783, 23) (2783,)
Kích thước tập kiểm thử: (696, 23) (696,)


In [218]:
print("Số lượng giá trị khuyết trong y sau khi loại bỏ:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong y sau khi loại bỏ:
0


In [219]:
import numpy as np

class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.left = None
        self.right = None
        self.threshold = None
        self.feature_index = None
        self.value = None

    def fit(self, X, y, depth=0):
        num_samples, num_features = X.shape
        if num_samples >= self.min_samples_split and (self.max_depth is None or depth < self.max_depth):
            best_split = self._get_best_split(X, y, num_samples, num_features)
            if best_split:
                self.feature_index, self.threshold = best_split
                indices_left = X[:, self.feature_index] < self.threshold
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                self.left = DecisionTreeRegressor(self.max_depth, self.min_samples_split)
                self.right = DecisionTreeRegressor(self.max_depth, self.min_samples_split)
                self.left.fit(X_left, y_left, depth + 1)
                self.right.fit(X_right, y_right, depth + 1)
        self.value = np.mean(y)

    def _get_best_split(self, X, y, num_samples, num_features):
        best_mse = float("inf")
        best_index = None
        best_threshold = None
        
        for feature_index in range(num_features):
            feature_values = X[:, feature_index]
            sorted_indices = np.argsort(feature_values)
            feature_values, labels = feature_values[sorted_indices], y[sorted_indices]
            
            for i in range(1, num_samples):
                if feature_values[i] == feature_values[i - 1]:
                    continue
                split_threshold = (feature_values[i] + feature_values[i - 1]) / 2
                left_labels, right_labels = labels[:i], labels[i:]
                mse = self._calculate_mse(left_labels, right_labels)
                
                if mse < best_mse:
                    best_mse = mse
                    best_index = feature_index
                    best_threshold = split_threshold
        
        return (best_index, best_threshold) if best_index is not None else None

    def _calculate_mse(self, left_labels, right_labels):
        left_var = np.var(left_labels) * len(left_labels)
        right_var = np.var(right_labels) * len(right_labels)
        return left_var + right_var

    def predict(self, X):
        if self.left is None or self.right is None:
            return np.array([self.value] * len(X))
        
        indices = X[:, self.feature_index] < self.threshold
        left_preds = self.left.predict(X[indices])
        right_preds = self.right.predict(X[~indices])
        
        return np.concatenate([left_preds, right_preds])



In [220]:
class RandomForestRegressor:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []
        if random_state is not None:
            np.random.seed(random_state)

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            bootstrap_indices = np.random.choice(len(X), size=len(X), replace=True)
            X_sample, y_sample = X[bootstrap_indices], y[bootstrap_indices]
            tree = DecisionTreeRegressor(self.max_depth, self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_predictions, axis=0)

In [221]:
from sklearn.ensemble import RandomForestRegressor

# Khởi tạo mô hình Random Forest
model = RandomForestRegressor(n_estimators=5000, random_state=42)  # Sử dụng 100 cây

# Huấn luyện mô hình trên tập huấn luyện
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=5000, random_state=42)

In [222]:
from sklearn.metrics import mean_squared_error, r2_score

# Dự đoán nhiệt độ trên tập kiểm thử
y_pred = model.predict(X_test)

# Đánh giá mô hình bằng Mean Squared Error và R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 4.9140819983109525
R-squared: 0.8636427107935002


In [223]:
weather

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,stations,sunrise_minutes,sunset_minutes,tempmax_next,cond_Clear,cond_Overcast,cond_Partially cloudy,cond_Rain,"cond_Rain, Overcast","cond_Rain, Partially cloudy"
0,Hà Nội,1990-01-01,15.9,13.4,14.6,15.9,13.4,14.6,14.0,96.3,...,48820099999,393,1046,17.4,False,True,False,False,False,False
1,Hà Nội,1990-01-02,17.4,14.6,16.1,17.4,14.6,16.1,13.7,85.9,...,48820099999,394,1046,18.2,False,True,False,False,False,False
2,Hà Nội,1990-01-03,18.2,16.1,16.8,18.2,16.1,16.8,13.8,82.5,...,48820099999,394,1047,19.6,False,True,False,False,False,False
3,Hà Nội,1990-01-04,19.6,15.2,16.4,19.6,15.2,16.4,14.3,88.2,...,48820099999,394,1048,18.1,False,True,False,False,False,False
4,Hà Nội,1990-01-05,18.1,15.4,16.9,18.1,15.4,16.9,15.8,93.8,...,48820099999,394,1048,17.7,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12533,Hà Nội,2024-04-25,34.0,25.0,28.9,45.6,25.0,34.5,25.9,84.7,...,"48820099999,48823099999,48825099999,4883109999...",330,1099,36.0,False,False,False,False,False,True
12534,Hà Nội,2024-04-26,36.0,27.0,30.9,47.9,31.0,39.6,26.9,80.8,...,VVNB,329,1099,40.0,False,False,True,False,False,False
12535,Hà Nội,2024-04-27,40.0,27.0,32.2,44.7,31.2,38.6,24.7,69.9,...,"48820099999,48823099999,48825099999,4883109999...",328,1100,35.0,True,False,False,False,False,False
12536,Hà Nội,2024-04-28,35.0,27.0,30.4,50.8,30.5,39.4,27.0,82.7,...,"48820099999,48823099999,48825099999,4883109999...",328,1100,34.0,False,False,True,False,False,False


In [224]:
data_30_4 = {
    'tempmax': [36],               # Nhiệt độ cao nhất
    'tempmin': [27],               # Nhiệt độ thấp nhất
    'humidity': [79.9],            # Độ ẩm
    'cloudcover': [40.2],          # Mây che phủ
    'windspeed': [22.3],           # Tốc độ gió
    'sealevelpressure': [999.1],   # Áp suất mực nước biển
    'moonphase' : [0.72],
    'visibility': [9] ,
    'winddir' : [82.5],
    'feelslikemax': [50] ,
    'feelslikemin': [31] ,
    'feelslike': [39.9], 
    'dew' : [26.9],
    'temp': [31],
    'windgust': [35.3],
    'sunrise_minutes': [326],
    'sunset_minutes': [1101],
    'cond_Clear': [0],             # Điều kiện trời quang đãng
    'cond_Overcast': [0],          # Điều kiện trời u ám
    'cond_Partially cloudy': [1],  # Điều kiện trời nhiều mây
    'cond_Rain': [0],              # Điều kiện mưa
    'cond_Rain, Overcast': [0],    # Điều kiện mưa và u ám
    'cond_Rain, Partially cloudy': [0],  # Điều kiện mưa và nhiều mây
    
}

new_data = pd.DataFrame(data_30_4)

In [225]:
new_data

,tempmax,tempmin,humidity,cloudcover,windspeed,sealevelpressure,moonphase,visibility,winddir,feelslikemax,...,temp,windgust,sunrise_minutes,sunset_minutes,cond_Clear,cond_Overcast,cond_Partially cloudy,cond_Rain,"cond_Rain, Overcast","cond_Rain, Partially cloudy"
0,36,27,79.9,40.2,22.3,999.1,0.72,9,82.5,50,...,31,35.3,326,1101,0,0,1,0,0,0


In [226]:
predicted_tempmax = model.predict(new_data)
print(f"Dự đoán nhiệt độ cao nhất cho ngày 1 tháng 5 tại Hà Nội là: {predicted_tempmax[0]}°C")

Dự đoán nhiệt độ cao nhất cho ngày 1 tháng 5 tại Hà Nội là: 33.1075400000002°C
